<a href="https://colab.research.google.com/github/stphnhng/482Project2/blob/master/Heuristic_%2B_Embedding_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
# Install the latest Tensorflow version.
!pip3 install --upgrade tensorflow-gpu
# Install TF-Hub.
!pip3 install tensorflow-hub
!pip3 install seaborn

#!mkdir encoder
#!curl -Lo encoder/infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
#!curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

In [2]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import json
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from nltk.tokenize import sent_tokenize

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [38]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
import spacy
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 826.9MB 1.3MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-cp36-none-any.whl size=828255078 sha256=e47ca0a2e8745ac74f36b3dfb9ad87b417ed591d9a67c73fe77849da8328ce7d
  Stored in directory: /tmp/pip-ephem-wheel-cache-lb5o4ekq/wheels/b4/d7/70/426d313a459f82ed5e06cc36a50e2bb2f0ec5cb31d8e0bdf09
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import en_core_web_lg
nlp = en_core_web_lg.load() 

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def read_json(file):
  with open(file, 'r') as fp:
    return json.loads(fp.read())

In [0]:
# Read in the data
data = read_json("/content/drive/My Drive/data_wInfo.json")
qa = read_json("/content/drive/My Drive/cp_alexa_qa.json")
generated_qa = read_json("/content/drive/My Drive/generated_qa.json")

In [0]:
stop_words = stopwords.words('english')

In [0]:
def parse_content(data):
  sentences = []
  outputs = []
  for section, content in data.items():
    if content['content'] != '':
      sent_toke = sent_tokenize(content['content'])
      for index, sent in enumerate(sent_toke):
        if sent == 'The W.K.':
          sent_toke[index+1] = sent + sent_toke[index+1]
        elif sent == 'Alumni include Abel Maldonado, Former California Lt.':
          sent_toke[index+1] = sent + sent_toke[index+1]
        else:
          word_list = nltk.word_tokenize(sent)
          output = [w for w in word_list if not w in stop_words]
          outputs.append(' '.join(output))
          sentences.append(sent)
  return sentences, outputs

In [0]:
sentences, outputs = parse_content(data)

In [0]:
for question in generated_qa:
    answer = generated_qa[question]
    if answer not in sentences:
        sentences.append(answer)

In [0]:
sents = []
for k in sentences:
  tok = nltk.word_tokenize(k.lower())
  ot = [word for word in tok if 'cal' not in word  and 'poly' not in word]
  sents.append(' '.join(ot))

In [0]:
sents.remove('official website athletics website',)

In [0]:
sentences_embeddings = embed(sents)

In [0]:
outputs_embeddings = embed(outputs)

In [0]:
def cos_sim(A, B):
  return np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

In [0]:
def match_generated(query):
    qa_generated_qs = list(generated_qa.items())
    most_similar_ans = ""
    most_similar_num = 0.0
    for q,answer in qa_generated_qs:
        removed_stopwords = [word for word in nltk.word_tokenize(q) if word not in stop_words]
        q = ' '.join(removed_stopwords)
        sim = nlp(query).similarity(nlp(q))
        if sim >= 0.75 and sim > most_similar_num:
            most_similar_ans = answer
            most_similar_num = sim
    return most_similar_num, most_similar_ans

In [0]:
def match_qa(query):
    qa_infobox_qs = list(qa.items())[:68] # >68 are infobox questions
    most_similar_ans = ""
    most_similar_num = 0.0
    for q,answer in qa_infobox_qs:
        removed_stopwords = [word for word in nltk.word_tokenize(q) if word not in stop_words]
        q = ' '.join(removed_stopwords)
        sim = nlp(query).similarity(nlp(q))
        if sim >= 0.75 and sim > most_similar_num:
            most_similar_ans = answer
            most_similar_num = sim
    return most_similar_num, most_similar_ans

In [0]:
def match_infobox(query):
    qa_infobox_qs = list(qa.items())[68:] # >68 are infobox questions
    most_similar_ans = ""
    most_similar_num = 0.0
    for q,answer in qa_infobox_qs:
        removed_stopwords = [word for word in nltk.word_tokenize(q) if word not in stop_words]
        q = ' '.join(removed_stopwords)
        sim = nlp(query).similarity(nlp(q))
        if sim >= 0.75 and sim > most_similar_num:
            most_similar_ans = answer
            most_similar_num = sim
    return most_similar_num, most_similar_ans

In [0]:
def find_sent_index(query, sentences_embeddings, min_sent_sim,  qa_match, info_match, gen_match):
  query_embedding = embed([query])
  max_sim_index = 0
  max_sim = -1
  sim_list = []
  for index, embedding in enumerate(sentences_embeddings):
    current_sim = cos_sim(query_embedding, embedding)[0]
    sim_list.append((index, current_sim))
    if current_sim > max_sim:
      max_sim = current_sim
      max_sim_index = index
  sim_list.sort(key=lambda x: x[1], reverse=True)
  if max_sim < min_sent_sim:
    qa_q_sim, qa_q = match_qa(query)
    # if qa_q_sim >= qa_match:
    #   print('qa_match')
    #   return -1, qa_q
    info_sim, info_q = match_infobox(query)
    if info_sim >= info_match:
      print(info_sim)
      print('info_match')
      return -1, info_q
    q_sim, generated_q = match_generated(query)
    if q_sim >= gen_match:
      print('generated_q match')
      return -1, generated_q
  return max_sim_index, sim_list

In [85]:
query = "Who is Ian Atol?"
index, scores = find_sent_index(query, sentences_embeddings, 0.3, 0.5, 0.8, 0.8)
if index == -1:
    print(scores)
else:
    top_sents = [nlp(sentences[i]) for i,score in scores[0:5]]
    embeddingResult = sentences[index]
    keywordResult = keywordExtractor(query, top_sents)
    embeddingSimilarity = nlp(query).similarity(nlp(embeddingResult))
    keywordSimilarity = nlp(query).similarity(nlp(keywordResult))
    ultimateResult = ""
    if embeddingSimilarity > keywordSimilarity:
        ultimateResult = embeddingResult
    else:
        ultimateResult = keywordResult
    print(ultimateResult)

qa_match
The live mascot, a living mustang, is named Chase, after Margaret Chase, the university's second president.


# Adding Keyword Extraction

In [0]:
generalized_pos = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'LS', 'MD', 'NN', 'PDT', 'POS',
   'PRP', 'RB', 'RP', 'SYM', 'TO', 'UH', 'VB', 'WDT', 'WP', 'WRB','SBAR', 'SINV', 'SQ', 
   'ADJP', 'ADVP', 'CONJP', 'FRAG', 'INTJ', 'LST', 
   'NAC', 'NP', 'NX', 'PP', 'PRN', 'PRT', 'QP', 'RRC', 'UCP', 'VP', 'WHADJP', 'WHADVP', 'WHNP', 
   'WHPP', 'X', 'S']
def getGeneralVersion(specific):
   if specific in generalized_pos:
      return specific
   for generalized in generalized_pos: 
      if generalized in specific:
         return generalized
   return specific

In [0]:
def keywordExtractor(query, topSents):
    # POS tag
    samp_str = nltk.pos_tag(nltk.word_tokenize(query))
    # Remove symbols and Cal Poly
    samp_str = [word+"_"+getGeneralVersion(tag) for word,tag in samp_str if tag != '.' and tag != 'WRB' and word != "Cal" and word != "Poly"]

    # Remove stopwords
    new_str = set()
    for word in samp_str:
        raw_word = word.split("_")[0]
        raw_tag = word.split("_")[1]
        if raw_word not in nlp.Defaults.stop_words:
            new_str.add(word)
            new_str.add(nlp(raw_word)[0].lemma_ + "_" + getGeneralVersion(raw_tag))
    # Adding synsets?
    for item in new_str:
        word = item.split("_")[0]
        tag = item.split("_")[1]
        word_syn = set()
        for syn in wordnet.synsets(word):
            word_syn.add(syn.name().split(".")[0] + "_" + tag)
            word_syn.add(nlp(word)[0].lemma_ + "_" + tag)
    # Add synonyms to word set for query
    for item in word_syn:
        new_str.add(item)
    keyword_top_sents = {}
    for nlp_sent in topSents:
        raw_sent = nlp_sent.text
        tokenized_pos = set()
        for word,tag in nltk.pos_tag(nltk.word_tokenize(raw_sent)):
            # TODO: change WRB to fit all Wh-Terms
            if tag != '.' and tag != 'WRB' and word != "Cal" and word != "Poly":
                # tokenized_pos.add(word+"_"+getGeneralVersion(tag))
                tokenized_pos.add(nlp(word)[0].lemma_ + "_" + getGeneralVersion(tag))
        keyword_top_sents[raw_sent] = tokenized_pos.intersection(new_str)
    for sent in {k: v for k, v in sorted(keyword_top_sents.items(), key=lambda item: item[1], reverse=1)}:
        return sent
        # print(len(keyword_top_sents[sent]), keyword_top_sents[sent], sent)

In [58]:
# POS tag
samp_str = nltk.pos_tag(nltk.word_tokenize("when were the red bricks completed ?"))
# Remove symbols and Cal Poly
samp_str = [word+"_"+getGeneralVersion(tag) for word,tag in samp_str if tag != '.' and tag != 'WRB' and word != "Cal" and word != "Poly"]

# Remove stopwords
new_str = set()
for word in samp_str:
    raw_word = word.split("_")[0]
    raw_tag = word.split("_")[1]
    if raw_word not in nlp.Defaults.stop_words:
        new_str.add(word)
        new_str.add(nlp(raw_word)[0].lemma_ + "_" + getGeneralVersion(raw_tag))

print(new_str)
# Apply stemming to match dictionary index terms (this doesn't exist for now)

{'bricks_NN', 'complete_VB', 'red_JJ', 'completed_VB', 'brick_NN'}


In [0]:
# Adding synsets?
for item in new_str:
    word = item.split("_")[0]
    tag = item.split("_")[1]
    word_syn = set()
    for syn in wordnet.synsets(word):
        word_syn.add(syn.name().split(".")[0] + "_" + tag)
        word_syn.add(nlp(word)[0].lemma_ + "_" + tag)

In [0]:
# Add synonyms to word set for query
for item in word_syn:
    new_str.add(item)

In [0]:
keyword_top_sents = {}
for nlp_sent in top_sents:
    raw_sent = nlp_sent.text
    tokenized_pos = set()
    for word,tag in nltk.pos_tag(nltk.word_tokenize(raw_sent)):
        # TODO: change WRB to fit all Wh-Terms
        if tag != '.' and tag != 'WRB' and word != "Cal" and word != "Poly":
            # tokenized_pos.add(word+"_"+getGeneralVersion(tag))
            tokenized_pos.add(nlp(word)[0].lemma_ + "_" + getGeneralVersion(tag))
    keyword_top_sents[raw_sent] = tokenized_pos.intersection(new_str)

In [62]:
for sent in {k: v for k, v in sorted(keyword_top_sents.items(), key=lambda item: item[1], reverse=1)}:
    print(len(keyword_top_sents[sent]), keyword_top_sents[sent], sent)
# TODO: lemmatize all word b/c comparison won't work for tenses

2 {'red_JJ', 'complete_VB'} The six "red-brick" halls were completed shortly afterward in 1959.
1 {'complete_VB'} The Sierra Madre and Yosemite halls were finished by 1968, and the Cerro Vista Apartments were completed in 2003.
0 set() Loans comprised 55.6% of the financial aid, 31.2% came in the form of grants, and 10.2% in scholarships.
0 set() The state legislature changed the school's official name again in 1971 to California Polytechnic State University, and since the 1970s the university has seen steady enrollment growth and building construction.
0 set() Cal Poly has two mascots.


# Optimization Stuff

In [0]:
import numpy as np

In [0]:
pos_scores = slice(0.3, 0.9, 0.3)

In [0]:
from scipy.optimize import minimize, fmin, brute

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

NameError: ignored

In [0]:
def fun_sim(params):
  print('************')
  print("params: ", *params)
  matches = 0
  total = len(qa)
  for i, (qs, ans) in enumerate(qa.items()):
    index, scores = find_sent_index(qs, sentences_embeddings, *params)
    if index == -1:
      sent = scores
    else:
      sent = sentences[index]
    if sent == ans:
      #print("Match")
      matches += 1
    #print("---------------")
    if i % 20 == 0 and i != 0:
      print("done: ", i)
  print('done: ', matches/total)
  return 1 - matches/total

In [0]:
result = minimize(fun_sim, [0.3, 0.5, 0.5, 0.5], method='Nelder-Mead')

In [0]:
resbrute = brute(fun_sim, (pos_scores, pos_scores, pos_scores, pos_scores),
                          full_output=True, finish=fmin, workers=-1)

# Testing

In [0]:
test_datv= {
    'When did the state legislature change the school\'s official name?': 'The state legislature changed the school\'s official name again in 1971 to California Polytechnic State University, and since the 1970s the university has seen steady enrollment growth and building construction.',
    'How many parking spaces does cal poly have?': 'In its most recent survey of available parking spaces on campus, the Cal Poly University Police reported 2,892 general purpose parking spaces, 3,492 dorm resident spaces, and 8,648 total spaces.',
    'Does cal poly require students to declare a major before the start of their freshman year?': 'Cal Poly requires students to declare a major when applying for admission, and the university then admits the most competitive applicants within each major based on GPA and SAT or ACT scores.',
    'When were the red bricks completed?' : 'The six "red-brick" halls were completed shortly afterward in 1959.',
    'How good is cal poly for latino student success?' : 'The Los Angeles Times reported that the university "was ranked one of the nation’s worst serving institutions for Latino student success."',
    'Who is kristin smart?' : 'On May 25, 1996, Cal Poly student Kristin Smart disappeared without a trace after a birthday party.',
    'What division does cal poly play college rugby in?' : 'The Mustangs play college rugby in the California conference of Division 1-A.',
    'What percent of slo residents live below the poverty line?' : 'Approximately 27% of the residents live below the poverty line, a similar percentage to Cleveland, Ohio.',
    'How much did poly canyon village cost?' : 'The Poly Canyon Village housing complex, with a similar style as the Cerro Vista apartments, was completed in 2009 at a cost of $300 million, making it the California State University system\'s largest construction project to date.',
    'How can you apply for a change of major, and what classes are required?':'Each major has a specific change of major plan which includes required classes to be taken while maintaining a certain GPA (usually between 2.5–2.75) in order to be considered as a candidate.',
    'How racially diverse is the student population compared to other California State Universities?': 'As of 2018, Cal Poly San Luis Obispo has the least racially diverse student population of all California State University and University of California campuses, though it is still far more diverse than the surrounding community it serves.',
    'What is the Week of Welcome program?' : 'The Week of Welcome program, more commonly known as "WOW", serves as a volunteer-based orientation program for new students during the first week after move-in during the beginning of the school year in September. Its purpose is to introduce students to the campus and the community and prepare them for a successful college career.',
    'When were the Greek organizations started?' : 'Greek organizations have been at Cal Poly since 1949.',
    'Which fraternity was suspended for wearing blackface?': 'In 2018, the Lambda Chi Alpha fraternity was suspended following a photo surfacing that showed a student wearing blackface.',
    'How many housing spaces are available?' : 'Cal Poly\'s on-campus student housing of 6,239 spaces',
    'How much is the average student’s tuition fee?' : 'Fall 2011 fees for the average student reached approximately $2,600 per quarter.',
    'What is the acceptance rate?' : 'For the class entering Fall 2018, 16,491 freshmen were accepted out of 54,663 applicants, a 30.2% acceptance rate, and 4,398 enrolled.',
    'What is the bus system called?' : 'The city\'s SLO Transit bus system provides service to and from campus.',
    'What percent of freshmen live on campus?' : 'Cal Poly housed 35.9% of fall 2015 undergraduates in 28 dorms on campus, and 98.7% of first-time freshmen lived on campus.',
    'When was Greek Life suspended?' : 'As of April 17, 2018, Greek Life has been suspended indefinitely by the university following a number of racial controversies, the most recent of which being the 2018 Cal Poly Blackface Incident.'
}

In [0]:
om = {}
for k,v in test_datv.items():
  tok = nltk.word_tokenize(k.lower())
  ot = [word for word in tok if 'cal' not in word  and 'poly' not in word]
  om[' '.join(ot)] = v

In [90]:
for q,a in om.items():
    index, scores = find_sent_index(q, sentences_embeddings, *[0.35, 0.7, 0.8, 0.8])
    query = q
    sent = ""
    if index == -1:
        sent = scores
        print(scores)
    else:
        top_sents = [nlp(sentences[i]) for i,score in scores[0:5]]
        embeddingResult = sentences[index]
        keywordResult = keywordExtractor(query, top_sents)
        embeddingSimilarity = nlp(query).similarity(nlp(embeddingResult))
        keywordSimilarity = nlp(query).similarity(nlp(keywordResult))
        ultimateResult = ""
        if embeddingSimilarity > keywordSimilarity:
            ultimateResult = embeddingResult
        else:
            ultimateResult = keywordResult
        sent = ultimateResult
    print('**********')
    print("Question: ", q)
    print('Predicted: ', sent)
    print('Actual: ', a)

**********
Question:  when did the state legislature change the school 's official name ?
Predicted:  The state legislature changed the school's official name again in 1971 to California Polytechnic State University, and since the 1970s the university has seen steady enrollment growth and building construction.
Actual:  The state legislature changed the school's official name again in 1971 to California Polytechnic State University, and since the 1970s the university has seen steady enrollment growth and building construction.
**********
Question:  how many parking spaces does have ?
Predicted:  In its most recent survey of available parking spaces on campus, the Cal Poly University Police reported 2,892 general purpose parking spaces, 3,492 dorm resident spaces, and 8,648 total spaces.
Actual:  In its most recent survey of available parking spaces on campus, the Cal Poly University Police reported 2,892 general purpose parking spaces, 3,492 dorm resident spaces, and 8,648 total spaces

## Test 2: QA questions - Hardcoded

In [0]:
om = {}
for k,v in qa.items():
  tok = nltk.word_tokenize(k)
  ot = [word for word in tok if 'cal' not in word.lower()  and 'poly' not in word.lower()]
  om[' '.join(ot)] = v

In [97]:
for q,a in om.items():
    index, scores = find_sent_index(q, sentences_embeddings, *[0.35, 0.7, 0.8, 0.8])
    query = q
    sent = ""
    if index == -1:
        sent = scores
        print(scores)
    else:
        top_sents = [nlp(sentences[i]) for i,score in scores[0:5]]
        embeddingResult = sentences[index]
        keywordResult = keywordExtractor(query, top_sents)
        embeddingSimilarity = nlp(query).similarity(nlp(embeddingResult))
        keywordSimilarity = nlp(query).similarity(nlp(keywordResult))
        ultimateResult = ""
        if embeddingSimilarity > keywordSimilarity:
            ultimateResult = embeddingResult
        else:
            ultimateResult = keywordResult
        sent = ultimateResult
    print('**********')
    print("Question: ", q)
    print('Predicted: ', sent)
    print('Actual: ', a)

**********
Question:  When was Founded ?
Predicted:  The corporation was founded in 1940 and was known as the Cal Poly Foundation until February 1, 2006.
Actual:  Cal Poly San Luis Obispo was established as the California Polytechnic School in 1901 when Governor Henry T. Gage signed the California Polytechnic School Bill after a campaign by journalist Myron Angel.
generated_q match
Cal Poly San Luis Obispo was established as the California Polytechnic School in 1901 when Governor Henry T. Gage signed the California Polytechnic School Bill after a campaign by journalist Myron Angel.
**********
Question:  Who founded ?
Predicted:  Cal Poly San Luis Obispo was established as the California Polytechnic School in 1901 when Governor Henry T. Gage signed the California Polytechnic School Bill after a campaign by journalist Myron Angel.
Actual:  Cal Poly San Luis Obispo was established as the California Polytechnic School in 1901 when Governor Henry T. Gage signed the California Polytechnic Sc